# Multivariate time series: data analysis and preprocessing

## Dataset

The data for this assignment contains time series for hourly weather parameters and pollution in the Chinese city of Bejing. The total data set contains 5 years of measurements, of which the last year is to be used as a test set.

In this notebook, we show how to import and visualise the raw data and demonstrate some useful ways of preprocessing in order to make the features more useful as inputs to a neural network. 

**Warning:** the provided feature transformations are plausible ones, for which we provide code to help students who are less proficient in coding. It is up to you to decide which features (original or transformed) you will use in your model(s).

No code for detrending is provided. As this is not the main focus of this assignment, we provide features (transformed versions of the time data) from which the network should be able to extract trends itself! Although this may not be ideal (more features = more complex network), this approach is good enough for the present assignment.

## Imports and Drive mounting


In [ ]:
#imports and Drive mounting
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd
from pandas import read_csv
from datetime import datetime
import math
import os

# fix random seed for reproducibility
seed = 2020
np.random.seed(seed)  

from sklearn.model_selection import train_test_split

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers


In [ ]:
# uncomment if you want to use Drive

#from google.colab import drive
#drive.mount('/content/gdrive')
#

##!ls '/content/gdrive/My Drive/'

## Reading the data set

The data is provided as a ".csv" file along with the getting started notebooks on Ufora. The easiest way to use it is to put it in your Drive folder and import it from there. 

For reading 'csv' files, using the pandas library is the easiest. This gives you back the data as a pandas dataframe. The code below imports the data and visualises the raw time series.


In [ ]:
# load dataset

# The PATH setting below assumes you just uploaded the data file to your Colab session
# When using Drive: replace this by the path where you put the data file
DATAPATH = './'  

DATAFILE = DATAPATH+'pollution_data.csv'
dataset = read_csv(DATAFILE, header=0, index_col=0)
dataset.head()



In [ ]:
values = dataset.values

# plotting the first k samples of the raw time series
# setting below for all samples
samples = values.shape[0]   
# setting below for smaller number of samples to "zoom in", e.g., 1000
#samples = 1000


print("Dataset has ",values.shape[0]," samples and ",values.shape[1], " features")
# specify columns to plot
groups = [0,1,2,3,4,5,6,7,8,9,10,11]
i = 1
# plot each column
plt.figure(figsize=(16,26))
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:samples, group])
    plt.title(dataset.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

## Discrete and categorical features

Most of the features are numeric. However, some of those numbers are discrete:
- year: 2010, 2011, 2012, 2013, 2014
- month: 1, ..., 12
- day: 1, ..., 31 (note that not all months have the same number of days)
- hour: 0, ..., 23

Wind directions are categorical values, coded as ['NE' 'NW' 'SE' 'cv'].

In order to decide what to do with those, we should think about the problem again (domain knowledge). 

**Year:** We need to use the year in order to extract the test set further on. Aside from that, there are no indications that the pollution is systematically rising, so it will probably not be useful for detrending or as an input feature to the model.

**Month:** There is a vague seasonal trend in the pollution, so knowing the month definitely could be useful. However, we can see clear seasonal trends in some of the wheather parameters (dew, temperature and pressure), so the month information may already be sufficiently present in these features. This means that in a first attempt, you could probably ignore the month feature.  

If you do use the month, for example if you feel the deviation of a feature from the monthly trend is relevant, its current 'encoding' does not match very well with the observed seasonal trends, which look like shifted sine signals. Below, we show how you can re-encode the months into sine and cosine signals, with which you can approximate any shifted sinusoidal signal with the same period. You could try to use these to detrend the 'periodic' features (trend is a linear combination of sine, cosine and a constant value). Either you can also detrend the labels, or you can input sine and cosine as separate features. 

**Day:** The day of the month does not seem very relevant to predict pollution, but the day of the week could be. Below, we provide code examples with which you can extract the day of the week and convert it to one-hot code in two ways: one way encodes each day separately, the other only discriminates between weekend and week days. In Belgium, the second option would be very relevant, but since we don't know commuter patterns in Bejing, it may be better to keep all days. You could also plot statistical data for each weekday to check whether there is any significant difference between different days of the week.

**Hour:** Time of day is probably relevant. You could use it as it is or you could consider a similar approach to what we did with the month: transforming into periodic sine and cosine signals and use these as features instead. Again, investigating the statistics for different times of day can help you decide.


In [ ]:
# Translating periodic discrete features into sines and cosines

monthp = values[:,1:2]*2.0*np.pi/12.0
month_sin = np.sin(np.array(monthp,dtype=np.float64))
month_cos = np.cos(np.array(monthp,dtype=np.float64))

#check - note that you would get a much smoother curve if you would use the day of the year
plt.figure(figsize=(16,8))
plt.subplot(4,1,1)
plt.plot(values[:,1:2])
plt.subplot(4,1,2)
plt.plot(month_sin)
plt.plot(month_cos)
plt.subplot(4,1,3)
plt.plot(values[:,6]) # plot temp for comparison

# We see that the transformed month curves are not very smooth. In fact, it is probably better to ignore the months 
# altogether and just use the day-of-the-year index to generate the periodic features
seasons = np.reshape(np.arange(values.shape[0])*2.0*np.pi/(365.0*24),(values.shape[0],1))
seasons_sin = np.sin(np.array(seasons,dtype=np.float64))
seasons_cos = np.cos(np.array(seasons,dtype=np.float64))

#check - note that you would get a much smoother curve if you would use the day of the year
plt.subplot(4,1,4)
plt.plot(seasons_sin)
plt.plot(seasons_cos)
plt.show()

In [ ]:
# Day of the week: take into account that 1/1/2010 was a Friday (you can look this up online)
days = np.zeros((values.shape[0],1))

# choose Monday to be day 0, then Friday is day 4
day = 4
for idx in range(values.shape[0]):
  days[idx,0] = day
  day = (day + 1) % 7 # modulo 7, so days are numbered 0-6

weekdays = np.ones_like(days)
weekdays[np.where(days>4)] = 0.0

mondays = np.zeros_like(days)
mondays[np.where(days==0)] = 1.0

## check
plt.figure(figsize=(16,6))
plt.subplot(3,1,1)
plt.plot(days[:100])
plt.subplot(3,1,2)
plt.plot(weekdays[:100])
plt.subplot(3,1,3)
plt.plot(mondays[:100])
plt.show()

We must now also convert the wind direction features to sensible numbers: possible options are integer, one-hot, 2D, polar, ...

From the original paper in which this dataset was introduced, we see that the three unique categories ['NE' 'NW' 'SE' 'cv'] are actually aggregates. The first three are bucketed according to the positions of large industrial sites around Bejing, whereas the fourth corresponds to "calm and variable": 

**Source:** https://royalsocietypublishing.org/doi/10.1098/rspa.2015.0257 :

**Quote:** "* ... northwest (NW), which includes W, WNW, NW, NNW and N; northeast (NE), for NNE, NE and ENE; southeast (SE), covering E, ESE, SE, SSE and S; southwest (SW), having SSW, SW and WSW; and calm and variable (CV).*"

From this perspective, one-hot seems to be the most sensible option for encoding these. This is implemented below, as well as a version in which the one-hot encoded vectors are multiplied by wind speed. 

Another option would be to use one-hot encoding for the three first options ('NE', 'NW', 'SE'), and spread out the third option across all three categories, so, for example, 'NE' would be represented by [1,0,0], whereas 'cv' would be [1/3,1/3,1/3] (or these values multiplied by wind speed).

In [ ]:
# Wind direction block

# creating two versions of converted feature sets for wind: 
# -  one-hot and wind speed separately (5 feature set) 
# -  4 feature set in which ones of one-hot have been replaced with wind speed


# show unique wind direction values
winddirs = np.unique(values[:,8])
print(winddirs)

dirindices = values[:,8]

encoded_winddir = np.zeros((len(dirindices),4))
encoded_dirstrength = np.zeros((len(dirindices),4))

for idx in range(4):
  indices = np.where(dirindices == winddirs[idx])
  encoded_winddir[indices,idx] = 1.0
  encoded_dirstrength[indices,idx] = values[indices,9]

# check
# Each block of two plots corresponds to a single wind direction
# # the top of both is the one-hot version, the botton is one-hot*strength

plt.figure(figsize=(16,16))
for idx in range(4):
    plt.subplot(8, 1, 2*idx+1)
    plt.plot(encoded_winddir[:1000, idx])
    plt.subplot(8, 1, 2*idx+2)
    plt.plot(encoded_dirstrength[:1000, idx])
plt.show()  

With the code above, we provide a range of possibly useful features. 

You will probably find that not all of these are in fact beneficial for your model. Since you do not have a lot of data and reducing the number of features is a major weapon against overfitting, you should definitely explore this. Obviously, if you get other (better) ideas, you can also generate new features yourself (and explain this in your report). 

After creating all features, you can join them together in a single feature matrix to use for training models. Ideally, you make a very rich feature set only once, and store it as a pandas dataframe. You can then use that as a starting point for your training notebooks.


In [ ]:
# helper function

def combine_features(feature_list):
  features = np.concatenate(feature_list,axis=1)
  return features

# the examples below show how you can select and combine the features you need
# NOTE that the selection made here is ONLY an example!!


# create 2 different wind feature sets
fwind4 = combine_features([encoded_dirstrength])
fwind5 = combine_features([values[:,9:10], encoded_winddir])

# create wheather feature set
fweather = values[:,5:8]

# extract pollution features
fpoll = values[:,4:5]

# extract precipitation features (rain/snow)
fwet = values[:,10:]


# Make a feature set of weather, periodic, weekdays, wind4
data=combine_features([fweather,seasons_cos,seasons_sin,weekdays,fwind4])


In [ ]:
# you can also add the new features to the pandas dataframe and save that for later reuse
# just a few features added here for demonstration purposes
dataset['cos']=seasons_cos
dataset['sin']=seasons_sin
dataset['weekdays']=weekdays
dataset['NE_strength']=encoded_dirstrength[:,0]
dataset['NW_strength']=encoded_dirstrength[:,1]
dataset['NS_strength']=encoded_dirstrength[:,2]
dataset['cv_strength']=encoded_dirstrength[:,3]

dataset.head()


In [ ]:
NEWDATAFILE = DATAPATH+'preprocessed_pollution_data.csv'
dataset.to_csv(path_or_buf=NEWDATAFILE)

#check
check = read_csv(NEWDATAFILE, header=0, index_col=0)
check.head() # should be exactly the same as dataset.head() above

newvalues=check.values

print(newvalues.shape)

labels = check.columns
print(labels)

## Splitting into train, validate and test sets

In order to avoid any problems with data leakage between train and test sets, it is safest to make the split from the very beginning. As stated: the last year is the test set, the year before that is the validation set.


In [ ]:
# The last year is meant for testing, the year of the data is in the first column
testyear = max(newvalues[:,0])
valyear = testyear - 1
print("Val set year is ", valyear)
print("Test set year is ", testyear)

# Find indices of all rows in test year
teststart = np.min(np.where(newvalues[:,0]==testyear)[0])
valstart = np.min(np.where(newvalues[:,0]==valyear)[0])
num_train_samples=valstart
num_val_samples = teststart - valstart
num_test_samples=newvalues.shape[0]-teststart

print("There are ",num_train_samples," samples in the training years,")
print(num_val_samples," samples in the validation year,")
print("and ",num_test_samples," samples in the test year")

# extract train and test data
train_data = pd.DataFrame(data=newvalues[:valstart,:],columns=labels)
TRAINDATAFILE = DATAPATH+'preprocessed_train_data.csv'
train_data.to_csv(path_or_buf=TRAINDATAFILE)

val_data = pd.DataFrame(data=newvalues[valstart:teststart,:],columns=labels)
VALDATAFILE = DATAPATH+'preprocessed_val_data.csv'
val_data.to_csv(path_or_buf=VALDATAFILE)

test_data = pd.DataFrame(data=newvalues[teststart:,:],columns=labels)
TESTDATAFILE = DATAPATH+'preprocessed_test_data.csv'
test_data.to_csv(path_or_buf=TESTDATAFILE)

#check
test_data.head()


This concludes our example code for data manipulation. Since data manipulation is not the core topic of this course, you are free to use this code as it is. However, whether you change it or not, hand in this notebook as well as the generated data files. If you do make changes, mention them in your report.




